In [1]:
!pip install transformers
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.8 MB/s eta 0:00:00


In [2]:
# Load the dataset
df_train = pd.read_csv('/content/drive/MyDrive/AML/sentiment_analysis/train.csv', encoding= 'unicode_escape' )
df_train = df_train.dropna()
df_test = pd.read_csv('/content/drive/MyDrive/AML/sentiment_analysis/test.csv', encoding= 'unicode_escape')
df_test = df_test.dropna()
# Split the dataset into train and test sets
train_texts,  train_labels = df_train['text'], df_train['sentiment']
test_texts,  test_labels = df_test['text'], df_test['sentiment']

In [3]:
# Instantiate the tokenizer and encode the texts
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

# Convert the labels to integers
label_map = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_map[label] for label in train_labels]
test_labels = [label_map[label] for label in test_labels]

In [4]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels)).shuffle(10000).batch(16)
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels)).batch(16)

In [6]:
# Instantiate the model
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)

# Define the optimizer and the loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Define the metrics
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]

# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
1718/1718 [==============================] - 738s 400ms/step - loss: 0.6152 - accuracy: 0.7423 - val_loss: 0.5738 - val_accuracy: 0.7612
Epoch 2/10
1718/1718 [==============================] - 702s 408ms/step - loss: 0.4563 - accuracy: 0.8195 - val_loss: 0.5322 - val_accuracy: 0.7756
Epoch 3/10
1718/1718 [==============================] - 681s 396ms/step - loss: 0.3338 - accuracy: 0.8747 - val_loss: 0.6423 - val_accuracy: 0.7646
Epoch 4/10
1718/1718 [==============================] - 680s 396ms/step - loss: 0.2285 - accuracy: 0.9185 - val_loss: 0.6444 - val_accuracy: 0.7719
Epoch 5/10
1718/1718 [==============================] - 681s 396ms/step - loss: 0.1652 - accuracy: 0.9436 - val_loss: 0.7172 - val_accuracy: 0.7606
Epoch 6/10
1718/1718 [==============================] - 700s 408ms/step - loss: 0.1328 - accuracy: 0.9548 - val_loss: 0.8054 - val_accuracy: 0.7612
Epoch 7/10
1718/1718 [==============================] - 700s 407ms/step - loss: 0.1074 - accuracy: 0.9653 - val_

In [7]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

# Generate predictions on the test set
test_preds = model.predict(test_dataset)
test_preds = np.argmax(test_preds.logits, axis=1)

# Print the classification report
class_names = ['negative', 'neutral', 'positive']
print(classification_report(test_labels, test_preds, target_names=class_names))

221/221 [==============================] - 24s 106ms/step - loss: 1.0023 - accuracy: 0.7586
Test Loss: 1.0022920370101929
Test Accuracy: 0.7586304545402527
221/221 [==============================] - 29s 103ms/step
              precision    recall  f1-score   support

    negative       0.73      0.80      0.76      1001
     neutral       0.76      0.68      0.72      1430
    positive       0.78      0.83      0.80      1103

    accuracy                           0.76      3534
   macro avg       0.76      0.77      0.76      3534
weighted avg       0.76      0.76      0.76      3534

